# 000 Forecasting Bot

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IFP import IFP, list_questions

In [3]:
ifps = list_questions()['results']

In [4]:
today_ids = list(sorted([x['id'] for x in ifps])) if ifps else [27238, 27239, 27240, 27241]
today_ids

[27238, 27239, 27240, 27241]

In [5]:
ifps = {id: IFP(id) for id in today_ids}

In [6]:
from Forecaster import Forecaster

In [7]:
fcst = Forecaster(ifps)

In [9]:
fcst.fit()

MODEL DOMAIN
27238 Will the USD be worth 0.935 EUR or more at the close of trading on any day before October 1, 2024?
Market price
27239 Will Keir Starmer's approval rating as Prime Minister of the United Kingdom be less than 40 on September 18, 2024?
Leadership change
27240 Will Keir Starmer's approval rating as Prime Minister of the United Kingdom be less than 50 on September 18, 2024?
Leadership change
27241 Will Keir Starmer's approval rating as Prime Minister of the United Kingdom be less than 60 on September 18, 2024?
Leadership change

CORRELATOR
|USD to EUR exchange rate|27238|Will the USD be worth 0.935 EUR or more at the close of trading on any day before October 1, 2024?|
|Keir Starmer's approval rating|27239|Will Keir Starmer's approval rating as Prime Minister of the United Kingdom be less than 40 on September 18, 2024?|
|Keir Starmer's approval rating|27240|Will Keir Starmer's approval rating as Prime Minister of the United Kingdom be less than 50 on September 18, 2024?|


In [10]:
fcst.report()

27238 Will the USD be worth 0.935 EUR or more at the close of trading on any day before October 1, 2024?
Forecast 75
Rationale The USD/EUR exchange rate has been fluctuating close to the 0.935 threshold, reaching 0.9321 EUR in the latest trading session. Given the ongoing market volatility and upcoming economic events like the Jackson Hole symposium and various central bank meetings, there is a substantial chance that the USD could briefly reach or exceed 0.935 EUR before October 1, 2024. Historical trends and analyst forecasts also suggest potential for significant short-term movements in response to economic data releases. Thus, I assign a 75% probability that the USD will be worth 0.935 EUR or more at the close of trading on any day before October 1, 2024. 

27239 Will Keir Starmer's approval rating as Prime Minister of the United Kingdom be less than 40 on September 18, 2024?
Forecast 75
Rationale Recent polls suggest a decreasing trend in Keir Starmer's approval rating, influenced

In [ ]:
fcst.upload()